# Python webpage parser - DigDirCamp 2020

Datasett generert 9 Juli 2020. 

Denne notebooken parser nettsidene til politiet for å lage eit datasett over lensmannskontor i Norge. Dette datasettet er dermed kun oppdatert pr 9 Juli 2020 og koden fungerer slik nettsidene til politiet såg ut den 9 Juli 2020.

## Import

In [ ]:
from lxml import html
import requests
import re
import pandas as pd
import numpy as np

## Globale variabler

In [ ]:
lat = []
lng = []
titl = []
stations = []
coordinates = []
site = []
pages = ['https://www.politiet.no/om/organisasjonen/politidistrikter/vest/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/ost/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/agder/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/finnmark/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/innlandet/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/more-og-romsdal/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/nordland/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/oslo/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/sor-vest/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/sor-ost/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/troms/',
         'https://www.politiet.no/om/organisasjonen/politidistrikter/trondelag/']

## Hent ut stasjonnavn, koordinater og link til nettside for alle politidistrikt

In [ ]:
for page in pages:
    website = requests.get(page)
    tree = html.fromstring(website.content)
    stations = np.append(stations, tree.xpath('/html/body/main/article/section/div[3]/ol/li/h3/a/text()'))
    coordinates = np.append(coordinates, tree.xpath('/html/body/main/article/section/div[3]/ol/li/div/a/@href'))
    site = np.append(site, tree.xpath('/html/body/main/article/section/div[3]/ol/li/h3/a/@href'))

## Konverter google maps link til flyttall + legg til tittelfelt for drupal

In [ ]:
for i in range(len(coordinates)):
    titl.append("Node " + str(i))
    result = re.findall(r"[-+]?\d*\.\d+|\d+", coordinates[i])
    lng.append(float(result[0]))
    lat.append(float(result[1]))

## Opprett dataframe

In [ ]:
d = {'title' : titl, 'field_stasjon': stations, 'field_politi_webside' : site, 'field_politi_lengdegrad': lat, 'field_politi_breddegrad' : lng}
df = pd.DataFrame(data=d)
df['field_politi_webside'] = 'https://www.politiet.no' + df['field_politi_webside'].astype(str)

## Lagre som CSV

In [ ]:
df.to_csv("politikontor.csv", index=False)